### TAREFAS

1- Crie um banco no SQLite utilizando o python, e salve nesse banco os dados do SCDE e os dados do cadastro.

In [188]:
import pandas as pd

In [189]:
df_scde = pd.read_csv('C:/Users/home/Downloads/case_comerc/scde_unidade_steste.csv', sep=';', encoding='latin1')
df_scde.head()

,Agente,Ponto / Grupo,Data,Hora,Ativa_C_kWh,Ativa_G_kWh,Reativa_C_kVArh,Reativa_G_kVArh,Qualidade,Origem
0,-,SPAJIVENTR-01 (L),01/11/2021,1,"4.958,80",0,"1.324,40",0,Completo,COLETA DIÁRIA
1,-,SPAJIVENTR-01 (L),01/11/2021,2,"4.720,80",0,"1.190,00",0,Completo,COLETA DIÁRIA
2,-,SPAJIVENTR-01 (L),01/11/2021,3,"4.597,60",0,"1.111,60",0,Completo,COLETA DIÁRIA
3,-,SPAJIVENTR-01 (L),01/11/2021,4,"4.986,80",0,"1.338,40",0,Completo,COLETA DIÁRIA
4,-,SPAJIVENTR-01 (L),01/11/2021,5,"5.336,80",0,"1.526,00",0,Completo,COLETA DIÁRIA


In [190]:
df_cadastro_1 = pd.read_excel('C:/Users/home/Downloads/case_comerc/Cadastro_teste.xlsx', sheet_name='cadastro1')
df_cadastro_1.head()

,COD,Unidades,Ponto_SCDE
0,000001-02,Unidade1,SPAJIVENTR-01 (L)
1,000108-15,Unidade6,PBITCPENTR101 (L)
2,000108-17,Unidade7,RSCMB1ENTR101 (L)
3,001168-06,Unidade4,SPHAEBENTR101 (L)
4,001168-08,Unidade5,SPHAVMENTR101 (L)


In [191]:
df_cadastro_2 = pd.read_excel('C:/Users/home/Downloads/case_comerc/Cadastro_teste.xlsx', sheet_name='cadastro2')
df_cadastro_2.head()

,COD,Grupo Econômico,Célula gestor,Fornecedor,Distribuidora
0,000001-02,GRUPO1,celula.cps01@comerc.com.br ...,Fornecedor3,CPFL PAULISTA
1,000108-15,GRUPO4,celula.cgc01@comerc.com.br ...,Fornecedor1,ENERGISA PARAIBA
2,000108-17,GRUPO4,celula.cgc01@comerc.com.br ...,Fornecedor3,RGE SUL
3,001168-06,GRUPO2,celula.SP02@comerc.com.br ...,Fornecedor3,ENEL SP
4,001168-08,GRUPO2,celula.SP02@comerc.com.br ...,Fornecedor3,ENEL SP


In [192]:
import sqlite3

In [193]:
conn = sqlite3.connect('banco_teste_infos.db')

In [194]:
df_scde.to_sql(name='scde', con=conn, index=False, if_exists='replace')
df_cadastro_1.to_sql(name='cadastro1', con=conn, index=False, if_exists='replace')
df_cadastro_2.to_sql(name='cadastro2', con=conn, index=False, if_exists='replace')

C:\Users\home\anaconda3\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [195]:
# Criando um cursor
cursor = conn.cursor()

# Executar a consulta para obter as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Buscando os resultados
tables = cursor.fetchall()

# Imprimir o nome das tabelas
for table in tables:
    print(table[0])

# Fechar a conexão
conn.close()

cadastro_unificado
scde
cadastro1
cadastro2


2- No SQLite, realize join das tabelas de cadastro numa única tabela nova.

In [196]:
# conectando com o banco
conn = sqlite3.connect('comerc_infos.db')

In [197]:
# Criando um cursor
cursor = conn.cursor()

In [198]:
# Junção cadastro1 & cadastro2
query = """SELECT 
              cadastro1.COD AS COD,
              cadastro1.Unidades AS Unidades,
              cadastro1.Ponto_SCDE AS Ponto_SCDE,
              cadastro2."Grupo Econômico" AS Grupo_Economico,
              cadastro2."Célula gestor" AS Celula_Gestor,
              cadastro2.Fornecedor AS Fornecedor,
              cadastro2.Distribuidora AS Distribuidora
            FROM cadastro1
            JOIN cadastro2 ON cadastro1.COD = cadastro2.COD;
"""

In [199]:
# Obter o resultado da query
result_df = pd.read_sql_query(query, conn)
print(result_df)

         COD   Unidades         Ponto_SCDE Grupo_Economico  \
0  000001-02   Unidade1  SPAJIVENTR-01 (L)          GRUPO1   
1  000108-15   Unidade6  PBITCPENTR101 (L)          GRUPO4   
2  000108-17   Unidade7  RSCMB1ENTR101 (L)          GRUPO4   
3  001168-06   Unidade4  SPHAEBENTR101 (L)          GRUPO2   
4  001168-08   Unidade5  SPHAVMENTR101 (L)          GRUPO2   
5  031060-04   Unidade3  SPFRP1ENTR101 (L)          GRUPO3   
6  031060-07   Unidade2  SPFRP2ENTR101 (L)          GRUPO3   
7  048168-21   Unidade9  SPTDPIENTR101 (L)          GRUPO6   
8  048168-24  Unidade10  SPTDRPENTR101 (L)          GRUPO6   
9  097181-01   Unidade8  SPSHTTALADM04 (L)          GRUPO5   

                                       Celula_Gestor   Fornecedor  \
0  celula.cps01@comerc.com.br                    ...  Fornecedor3   
1  celula.cgc01@comerc.com.br                    ...  Fornecedor1   
2  celula.cgc01@comerc.com.br                    ...  Fornecedor3   
3  celula.SP02@comerc.com.br             

In [200]:
# Salvando o resultado como uma tabela no banco de dados
result_df.to_sql(name='cadastro_unificado', con=conn, index=False, if_exists='replace')

In [201]:
# Criando um cursor
cursor = conn.cursor()

# Executar a consulta para obter as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Buscando os resultados
tables = cursor.fetchall()

# Imprimir o nome das tabelas
for table in tables:
    print(table[0])

# Fechar a conexão
conn.close()

scde
cadastro1
cadastro2
cadastro_unificado
